# 区块

## 定义Block

In [1]:
# 继承tuple
class Block(tuple):
    
    # version：版本号，prev_block_hash：前区块哈希值，timestamp：时间戳，bits：“挖矿”难度位数，nonce：工作量证明，txs：交易集合
    def __new__(cls,version,
                prev_block_hash,
                timestamp,
                bits,
                nonce,
                txs):
        return super(Block,cls).__new__(cls,(version,
                                             prev_block_hash,
                                             timestamp,
                                             bits,
                                             nonce,
                                             txs))
        
    # 版本类型
    @property
    def version(self):
        return self[0]
    
    # 前区块哈希值
    @property
    def prev_block_hash(self):
        return self[1]
    
    # 时间戳
    @property
    def timestamp(self):
        return self[2]
    
    # 难度位数
    @property
    def bits(self):
        return self[3]
    
    # 工作量证明
    @property
    def nonce(self):
        return self[4]
    
    #交易集合
    @property
    def txs(self):
        return self[5]

    # 梅克尔树根哈希值
    @property
    def merkle_root_hash(self):
        return self.get_merkle_root()
    
    # 创建一个新的Block，仅替换nonce
    def _replace(self,nonce = 0):
        return Block(self[0],
                     self[1],
                     self[2],
                     self[3],
                     nonce,
                     self[5])

    # 计算梅克尔树根哈希值
    def get_merkle_root(self):
        return get_merkle_root_of_txs(self.txs) if self.txs else None
    
    # 区块头，nonce：工作量证明、merkle_root_hash：梅克尔树根哈希值
    # 返回：版本、前区块哈希值、时间戳、难度位数、梅克尔树根哈希值、工作量证明的组合
    def header(self,nonce = None,merkle_root_hash = None):
        if merkle_root_hash is None:
            merkle_root_hash = self.get_merkle_root()
            
        return "{0}{1}{2}{3}{4}{5}".format(self[0],
                                           self[1],
                                           self[2],
                                           self[3],
                                           merkle_root_hash,
                                           nonce or self[4])
    
    # 区块的哈希值，可当作区块的编号
    # 对区块头数据进行哈希运算
    @property
    def hash(self):
        return sha256d(self.header())

    def __repr__(self):
        return "Block(hash:{0})".format(self.hash)

关于Simchain中的区块数据结构，示例如下：

In [2]:
from simchain import Network
net = Network()
# 第一个节点命名为张三
zhangsan = net.peers[0]
# 随机交易
net.make_random_transactions()
# 达成共识
net.consensus()
# 获取创世区块
gen_block = zhangsan.blockchain[0]
# 获取第2区块
block = zhangsan.blockchain[1]
# 区块版本
block.version

2019-06-12 21:07:20,525 - A blockchain p2p network created,12 peers joined
2019-06-12 21:07:20,570 - genesis block has been generated
2019-06-12 21:07:20,612 - peer(63, 34)(pid=5) created a transaction
2019-06-12 21:07:20,614 - peer(63, 34)(pid=5) sent a transaction to network
2019-06-12 21:07:21,395 - peer(63, 34)(pid=5)'s transaction verified by 11 peers
2019-06-12 21:07:21,442 - peer(45, 39)(pid=7) created a transaction
2019-06-12 21:07:21,442 - peer(45, 39)(pid=7) sent a transaction to network
2019-06-12 21:07:22,275 - peer(45, 39)(pid=7)'s transaction verified by 11 peers
2019-06-12 21:07:22,330 - peer(31, 67)(pid=6) created a transaction
2019-06-12 21:07:22,331 - peer(31, 67)(pid=6) sent a transaction to network
2019-06-12 21:07:23,151 - peer(31, 67)(pid=6)'s transaction verified by 11 peers
2019-06-12 21:07:23,194 - peer(18, 60)(pid=4) created a transaction
2019-06-12 21:07:23,195 - peer(18, 60)(pid=4) sent a transaction to network
2019-06-12 21:07:24,061 - peer(18, 60)(pid=4)'s

0

In [3]:
# 前区块哈希值
block.prev_block_hash

'24b9793a14e7c918b285c4a4f30cc42d3b7fe01f6f068662cf715a6cb5535d74'

In [4]:
# 创世区块哈希值
gen_block.hash

'24b9793a14e7c918b285c4a4f30cc42d3b7fe01f6f068662cf715a6cb5535d74'

In [5]:
# 梅克尔树根哈希值
block.merkle_root_hash

'0074acb96225fac9820a3ba2b780d8a396bb7762ce075b010af212e5b34d54b9'

In [6]:
# 时间戳
block.timestamp

0

In [7]:
# 挖矿难度位数
block.bits

18

In [8]:
# 解题答案，也就是工作量证明
block.nonce

5386

In [9]:
# 区块哈希值（编号）
block.hash

'00003657e8da9b6bf9e7a099f194b58418ace9c6881c630937004ff58e39ce70'

In [10]:
# 区块头
block.header

<bound method Block.header of Block(hash:00003657e8da9b6bf9e7a099f194b58418ace9c6881c630937004ff58e39ce70)>

In [11]:
# 区块交易
block.txs

[Tx(id:522c78c5b467ba927bd89425d8a692af804494343fdccb811f2764b387e49e66),
 Tx(id:7c5ba970219bd389fe7d07a76fcaa3a6e3ea449fa2e71848c0707893241b27cd),
 Tx(id:3824a25b3d3616a776503502b7e21ad73b237be46835b8bcaf131c62a2f61fbe),
 Tx(id:87b0d55543e412dda62af45faa1e2e03f1fb34723bf14bb09d68b8abb2ed57a2),
 Tx(id:80ef903667f4a0c9b1e131965fe6404032e658e5beebd21c3e44c7806d363d93)]

区块头并不直接包括交易数据，依靠梅克尔树根哈希值来防止交易数据被篡改。

In [12]:
# 删除区块交易中的最后一条交易
block.txs.pop()
# 区块的哈希值发生变化
block

Block(hash:9e1517df8ab1c65cb39cd38ba13496f28f6f4ee697481caf7decc3e620b7a8d7)

In [13]:
# 梅克尔树根哈希值也发生了变化
block.merkle_root_hash

'dda794e38f1805fa528251c3377abd2b20abf9424c1e0abeddc5d702ba229852'

计算交易的梅克尔树根哈希值的代码，如下所示：

In [14]:
# 计算交易的梅克尔树根哈希值
def get_merkle_root_of_txs(txs):
    return get_merkle_root([tx.id for tx in txs])

# 通过交易哈希值列表计算梅克尔树根哈希值
def get_merkle_root(level):

    # 当哈希值列表的长度为1时退出循环
    while len(level) != 1:
        # 初始化哈希值列表奇数长度时的末尾哈希值
        odd = None
        
        # 如果哈希值列表长度为奇数，则将末尾哈希值从列表中移除并存储到odd
        if len(level) % 2 == 1:
            odd = level.pop()
            
        # 对当前层级列表中的元素两两串联进行哈希运算并得到新的层级
        level = [sha256d(i1+i2) for i1,i2 in pair_node(level)]

        # 如果odd不为None，则添加到新层级中
        if odd:
            level.append(odd)
    return level[0]
    
# 返回列表节点的两两配对
def pair_node(l):
    return (l[i:i + 2] for i in range(0, len(l), 2))

In [15]:
from simchain import Pointer, Vin, Vout, Tx, Block, sha256d
p = Pointer(1, 2)
vout = Vout(1, 2)
vin = Vin(p, b'1', b'12')
tx = Tx([vin], [vout])
block = Block(1, 2, 3, 4, 5, [tx, tx])
tx

Tx(id:02fb6db812bb4183bfa8161c2d8fa43db82704b1470342ffce4868d3e1902060)

In [16]:
# 计算交易列表的梅克尔树根哈希值
sha256d(tx.id + tx.id)

'19f333840f20518ced9efdb63ef0e919382fd121fcaff457c8007e82a1eab9a3'

In [17]:
# 获取区块的梅克尔树根哈希值
block.merkle_root_hash

'19f333840f20518ced9efdb63ef0e919382fd121fcaff457c8007e82a1eab9a3'

In [18]:
# 定义新的区块
block = Block(1, 2, 3, 4, 5, [tx, tx, tx])
# 获取新区块的梅克尔树根哈希值
block.merkle_root_hash

'6551df1dad93c31607471b29faf6ce12942dfbe97918265fdd801914c89132ec'

In [19]:
h12 = sha256d(tx.id + tx.id)
# 计算交易列表的梅克尔树根哈希值
sha256d(h12 + tx.id)

'6551df1dad93c31607471b29faf6ce12942dfbe97918265fdd801914c89132ec'

## 梅克尔树MerkelTree

In [20]:
from simchain.ecc import sha256d

# 节点对象
class Node(object):

    # 初始输入data为哈希值或哈希值组合
    def __init__(self, data, prehashed=False):
        # 如果为底层节点，则节点哈希值为输入值data
        if prehashed:
            self.val = data
        # 如果为非底层节点，对data进行双哈希运算
        else:
            self.val = sha256d(data)
        
        # 左子节点
        self.left_child = None
        # 右子节点
        self.right_child = None
        # 父节点
        self.parent = None
        # 兄弟节点
        self.bro = None
        # 节点的方向，左或右
        self.side = None

    def __repr__(self):
        return "MerkleTreeNode('{0}')".format(self.val)

# 梅克尔树对象
class MerkleTree(object):
    
    # 初始参数为底层节点的哈希值列表
    def __init__(self,leaves = []):
        # 将哈希值存储在节点中，Node对象对的prehashed为True
        self.leaves = [Node(leaf,True) for leaf in leaves]
        # 初始化根节点
        self.root = None

    # 添加一个新节点，输入为节点哈希值
    def add_node(self,leaf):
        self.leaves.append(Node(leaf))

    # 梅克尔树清零
    def clear(self):
        self.root = None
        for leaf in self.leaves:
            leaf.parent,leaf.bro,leaf.side = (None,)*3

    # 计算梅克尓树根哈希值
    def get_root(self):
        # 如果底层节点哈希列表为空，返回None
        if not self.leaves:
            return None

        # 初始化底层列表
        level = self.leaves[::]
        
        # 如果计算层哈希列表长度为1，退出循环
        while len(level) != 1:
            # 构建新的层级
            level = self._build_new_level(level)
        self.root = level[0]
        return self.root.val
    
    # 构建新的层级
    def _build_new_level(self, leaves):
        
        # 初始化新层级节点列表，以及当前层节点数量为奇数时的末尾节点
        new, odd = [], None
        
        # 如果当前层节点数量为奇数，则将末尾节点从节点列表中移除
        # 并保存在odd中
        if len(leaves) % 2 == 1:
            odd = leaves.pop(-1)
            
        # 对节点哈希值两两组合
        for i in range(0, len(leaves), 2):
            # 生成父节点
            newnode = Node(leaves[i].val + leaves[i + 1].val)
            # 给父节点的左、右子节点赋值
            newnode.lelf_child, newnode.right_child = leaves[i], leaves[i + 1]
            # 给配对节点的方向赋值
            leaves[i].side, leaves[i + 1].side,  = 'LEFT', 'RIGHT'
            # 给配对节点的父节点赋值
            leaves[i].parent, leaves[i + 1].parent = newnode, newnode
            # 配对节点互为兄弟节点
            leaves[i].bro, leaves[i + 1].bro = leaves[i + 1], leaves[i]
            # 将新节点添加到新层级列表中
            new.append(newnode)
        
        # 如果odd不为空，将其添加到新层级列表的末尾
        if odd:
            new.append(odd)
        return new

    # 获取由底层节点计算根哈希所需要的所有哈希值，index为底层节点在列表中的索引
    def get_path(self, index):
        # 初始化路径
        path = []
        # 获取索引对应的底层节点
        this = self.leaves[index]
        # 将底层节点哈希值添加到路径
        path.append((this.val, 'SELF'))
        # 寻找梅克尔树路径
        while this.parent:
            # 将节点的兄弟节点哈希值添加到路径
            path.append((this.bro.val, this.bro.side))
            # 用父节点代替当前节点
            this = this.parent
        # 将根哈希值添加至路径
        path.append((this.val, 'ROOT'))
        return path

In [21]:
from simchain import MerkleTree, sha256d
# 底层节点哈希值列表
leaves = ['a', 'b', 'c', 'd']
# 创建梅克尔树对象
merkle = MerkleTree(leaves)
# 获取梅克尔树根哈希值
merkle.get_root()

'20c12afdb2ce90da744e7f06424176c0c36f633be6cadd4eeafcda65855a7a73'

In [22]:
# 计算索引为0的交易的梅克尔树路径
path = merkle.get_path(0)
path

[('a', 'SELF'),
 ('b', 'RIGHT'),
 ('032a3987db0858e6d3ebad8580da9f28774bd9e0158e78d6ef42a68869750e26', 'RIGHT'),
 ('20c12afdb2ce90da744e7f06424176c0c36f633be6cadd4eeafcda65855a7a73', 'ROOT')]

In [23]:
# 根据路径计算根哈希值
ab = sha256d(path[0][0] + path[1][0])
abcd = sha256d(ab + path[2][0])
abcd

'20c12afdb2ce90da744e7f06424176c0c36f633be6cadd4eeafcda65855a7a73'

In [24]:
assert abcd == path[3][0]

还可以从区块中获取交易的梅克尔树路径

In [25]:
from simchain import Network
net = Network()
net.make_random_transactions()
net.consensus()
zhangsan, lisi = net.peers[0], net.peers[4]

2019-06-12 21:07:41,835 - A blockchain p2p network created,12 peers joined
2019-06-12 21:07:41,835 - genesis block has been generated
2019-06-12 21:07:41,886 - peer(1, 92)(pid=11) created a transaction
2019-06-12 21:07:41,888 - peer(1, 92)(pid=11) sent a transaction to network
2019-06-12 21:07:42,680 - peer(1, 92)(pid=11)'s transaction verified by 11 peers
2019-06-12 21:07:42,711 - peer(95, 83)(pid=3) created a transaction
2019-06-12 21:07:42,727 - peer(95, 83)(pid=3) sent a transaction to network
2019-06-12 21:07:43,520 - peer(95, 83)(pid=3)'s transaction verified by 9 peers
2019-06-12 21:07:43,570 - peer(66, 96)(pid=1) created a transaction
2019-06-12 21:07:43,571 - peer(66, 96)(pid=1) sent a transaction to network
2019-06-12 21:07:44,461 - peer(66, 96)(pid=1)'s transaction verified by 11 peers
2019-06-12 21:07:44,504 - peer(66, 96)(pid=1) created a transaction
2019-06-12 21:07:44,506 - peer(66, 96)(pid=1) sent a transaction to network
2019-06-12 21:07:45,405 - peer(66, 96)(pid=1)'s 

In [26]:
# 获取李四区块链中第2区块的交易列表
txs = lisi.blockchain[1].txs
# 创建梅克尔树
merkle = MerkleTree([tx.id for tx in txs])
# 梅克尔树根哈希值
merkle.get_root()

'd0bea78bb3d2c3388d5eff53198e348c6438903925303c09544a905ec4902dd7'

In [27]:
lisi.blockchain[1].merkle_root_hash

'd0bea78bb3d2c3388d5eff53198e348c6438903925303c09544a905ec4902dd7'

In [28]:
# 获取索引为0交易的梅克尔树路径
merkle.get_path(0)

[('d43f6bacef3b5ed48febe98f303f19dfc8546b19209a5e5a8e152b62a0b161db', 'SELF'),
 ('3ea39db4b45bb1dcea3fbec28c84b9bff3e8bab8d8f5fe271595e1d1438d1984', 'RIGHT'),
 ('2f6e2fee71ee587806c68d943af734d64efebcee7e412bd617e22cfc826ffb97', 'RIGHT'),
 ('fb2439d1d61415a40744c7b2dd566d2eed7b02234f67e3139f65b433d1a3704b', 'RIGHT'),
 ('d0bea78bb3d2c3388d5eff53198e348c6438903925303c09544a905ec4902dd7', 'ROOT')]